<a href="https://colab.research.google.com/github/erindakapllani/AppliedMath_Task2/blob/main/NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import os

In [17]:
import spacy
from spacy.matcher import Matcher

In [18]:
# Load the SpaCy model
nlp = spacy.load("en_core_web_sm")

In [19]:
# Define the path to the folder containing the documents
folder_path = '/content/Nuclear_safety'

In [20]:
# Function to read all text files from the folder
def read_documents(folder_path):
    documents = []
    for filename in os.listdir(folder_path):
        if filename.endswith('.txt'):
            with open(os.path.join(folder_path, filename), 'r', encoding='utf-8') as file:
                documents.append(file.read())
    return documents

In [21]:
# Function to generate questions based on entities
def generate_questions(doc):
    qa_pairs = []
    for ent in doc.ents:
        if ent.label_ in ["PERSON", "ORG", "GPE", "DATE", "TIME", "MONEY", "PERCENT"]:
            question = f"What is {ent.text}?"
            answer = ent.sent.text
            qa_pairs.append((question, answer))
        elif ent.label_ in ["EVENT", "WORK_OF_ART", "LAW"]:
            question = f"What is {ent.text} about?"
            answer = ent.sent.text
            qa_pairs.append((question, answer))
    return qa_pairs

In [22]:
# Function to generate questions based on nuclear-specific patterns
def generate_rule_based_questions(doc):
    matcher = Matcher(nlp.vocab)

    # Define patterns specific to the nuclear industry
    patterns = [
        [{"LOWER": "nuclear"}, {"LOWER": "reactor"}],
        [{"LOWER": "nuclear"}, {"LOWER": "fission"}],
        [{"LOWER": "uranium"}],
        [{"LOWER": "pressurized"}, {"LOWER": "water"}, {"LOWER": "reactor"}],
        [{"LOWER": "safety"}, {"LOWER": "measures"}],
        [{"LOWER": "radiation"}, {"LOWER": "leaks"}],
        [{"LOWER": "international"}, {"LOWER": "atomic"}, {"LOWER": "energy"}, {"LOWER": "agency"}],
    ]

    # Adding patterns to the matcher
    for i, pattern in enumerate(patterns):
        matcher.add(f"NUCLEAR_PATTERN_{i}", [pattern])

    # Processing the document and extracting QA pairs
    qa_pairs = []
    matches = matcher(doc)
    for match_id, start, end in matches:
        span = doc[start:end]
        question = f"What is {span.text}?"
        answer = span.sent.text
        qa_pairs.append((question, answer))

    return qa_pairs


In [23]:
# Main function to process documents and generate QA pairs
def main():
    documents = read_documents(folder_path)
    all_qa_pairs = []

    for document in documents:
        doc = nlp(document)
        qa_pairs = generate_rule_based_questions(doc)
        all_qa_pairs.extend(qa_pairs)

    # Output the QA pairs
    for question, answer in all_qa_pairs:
        print(f"Question: {question}\nAnswer: {answer}\n")

In [24]:
# Run the main function
if __name__ == "__main__":
    main()

FileNotFoundError: [Errno 2] No such file or directory: '/content/Nuclear_safety'